<a href="https://colab.research.google.com/github/qngyn/BigData_Project1/blob/main/Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 48.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c295b378a3b4c86285874ddb7138caa1a457e797810ba5d22dd602884818a92f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
pip install pyodbc 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 472 kB 4.0 MB/s 


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyodbc 
import pandas as pd 
import pyspark.sql.functions as sf 

In [4]:
spark = SparkSession.builder.config('spark.driver.memory', '16g').getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
def process_log_data(path,file_name,date):
    df = spark.read.json(path+file_name)
    df = df.select('_source.AppName','_source.Contract','_source.Mac','_source.TotalDuration')
    df = df.withColumn('Date',lit(date))
    df = df.withColumn('Type',
                 when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
          .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
                 (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
          .when((col("AppName") == 'RELAX'), "Giải Trí")
          .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
          .when((col("AppName") == 'SPORT'), "Thể Thao")
          .otherwise("Error"))
    df = df.select('Contract','Type','TotalDuration','Date')
    df = df.groupBy('Contract','Type','Date').agg({'TotalDuration':'sum'}).withColumnRenamed('sum(TotalDuration)','TotalDuration')
    return df 

In [49]:
path ='/content/drive/MyDrive/Dataset/'
file_name='20220401.json'
date = '2022-04-01'
df = process_log_data(path, file_name, date)

In [50]:
df.show()

+---------+-----------+----------+-------------+
| Contract|       Type|      Date|TotalDuration|
+---------+-----------+----------+-------------+
|HNJ141458|Truyền Hình|2022-04-01|        47349|
|SGJ147370|Truyền Hình|2022-04-01|        83331|
|HNFD80728|Truyền Hình|2022-04-01|        91636|
|SGH574803|Truyền Hình|2022-04-01|       113492|
|NTFD92901|Truyền Hình|2022-04-01|        18944|
|TBD015606|Phim Truyện|2022-04-01|        20586|
|AGFD54302|Truyền Hình|2022-04-01|        11860|
|SGH919014|Truyền Hình|2022-04-01|        55429|
|NTFD63152|Truyền Hình|2022-04-01|        18886|
|BTFD15886|Phim Truyện|2022-04-01|         6007|
|HND095909|Phim Truyện|2022-04-01|        16441|
|SGH209300|Truyền Hình|2022-04-01|        25527|
|NND000787|Truyền Hình|2022-04-01|          175|
|DNFD68426|Phim Truyện|2022-04-01|         3563|
|CBFD05588|Truyền Hình|2022-04-01|        62005|
|HTFD01984|Truyền Hình|2022-04-01|        81976|
|GLD014406|Truyền Hình|2022-04-01|        81937|
|DNAAA1260|Truyền Hì

In [51]:
df.count()

1483421

In [52]:
file_name2 = '20220402.json'
date2 ='2022-04-02'
df2 = process_log_data(path, file_name2, date2)
df2.show()

+---------+-----------+----------+-------------+
| Contract|       Type|      Date|TotalDuration|
+---------+-----------+----------+-------------+
|BTFD46314|Phim Truyện|2022-04-02|        12905|
|TID018269|Phim Truyện|2022-04-02|        10602|
|THFD16957|Phim Truyện|2022-04-02|        22486|
|QNH003340|   Thể Thao|2022-04-02|         1282|
|TIFD11370|Phim Truyện|2022-04-02|        16514|
|HDFD66334|Truyền Hình|2022-04-02|          311|
|HNH882924|Truyền Hình|2022-04-02|        86400|
|DAH000931|   Thể Thao|2022-04-02|          580|
|QBFD05730|Phim Truyện|2022-04-02|         3411|
|QNFD63536|Phim Truyện|2022-04-02|        53822|
|THFD08735|Truyền Hình|2022-04-02|        11614|
|HNH694160|Truyền Hình|2022-04-02|          299|
|HPFD84528|Truyền Hình|2022-04-02|          299|
|LAFD28252|Truyền Hình|2022-04-02|          299|
|SGH801986|Truyền Hình|2022-04-02|          288|
|HNH628036|Truyền Hình|2022-04-02|        86400|
|HYFD01337|   Thể Thao|2022-04-02|           72|
|DAAAA3937|Phim Truy

In [53]:
df2.count()

1470098